<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Merging-main-files" data-toc-modified-id="Merging-main-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Merging main files</a></span></li><li><span><a href="#Loading-primers" data-toc-modified-id="Loading-primers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Loading primers</a></span></li><li><span><a href="#DairyDB" data-toc-modified-id="DairyDB-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DairyDB</a></span></li></ul></div>

## Packages

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## DairyDB

In [124]:
from processing.hvr_extraction import get_all_hvr
from loading.main_sequence_loading import main_loading
from processing.hvr_statistics import main_stats

In [127]:
df_DB_DB = main_loading('DairyDB')

In [ ]:
hvr_df_DB_DB = get_all_hvr(df, primers_origin='DairyDB', save_name='hvr_DB_sequences_DB_primers')

In [128]:
main_stats(df_DB_DB, hvr_df_DB_DB, sequence_origin='DairyDB', primers_origin='DairyDB')

-------------------------------------------------- GLOBAL INFORMATION --------------------------------------------------
Total number of observations: 10389
Average sequence size: 1446.56
Standard deviation: 67.53
Minimal size: 933
Maximal size: 2229
------------------------------------------------- TAROXOMY INFORMATION -------------------------------------------------
------------------------------------------------------- KINGDOM --------------------------------------------------------
Total number of unique kingdom: 2
Rank 1 in kingdom: BACTERIA with 10282 occurences
Rank 2 in kingdom: ARCHAEA with 107 occurences
0 missing values
-------------------------------------------------------- PHYLUM --------------------------------------------------------
Total number of unique phylum: 46
Rank 1 in phylum: FIRMICUTES with 3917 occurences
Rank 2 in phylum: PROTEOBACTERIA with 2310 occurences
Rank 3 in phylum: BACTEROIDETES with 1473 occurences
Rank 4 in phylum: ACTINOBACTERIA with 935 occur

Total number of remaining kingdom with at least one V1 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V2 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V3 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V4 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V5 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V6 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V7 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V8 found: 1 - 50.0% of total number of groups
Total number of remaining kingdom with at least one V9 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V12 found: 1 - 50.0% of total number of groups
Total number of r

Total number of remaining family with at least one V34 found: 396 - 86.0% of total number of groups
Total number of remaining family with at least one V45 found: 377 - 82.0% of total number of groups
Total number of remaining family with at least one V56 found: 401 - 87.0% of total number of groups
Total number of remaining family with at least one V67 found: 374 - 81.0% of total number of groups
Total number of remaining family with at least one V78 found: 420 - 91.0% of total number of groups
Total number of remaining family with at least one V89 found: 312 - 68.0% of total number of groups
-------------------------------------------------------- GENUS ---------------------------------------------------------
Total number of unique genus: 1754
0 missing values - 0.0% of total
Total number of remaining genus with at least one V1 found: 648 - 37.0% of total number of groups
Total number of remaining genus with at least one V2 found: 1167 - 67.0% of total number of groups
Total number o